In [ ]:
from dotenv import load_dotenv
from binance.client import Client
from datetime import datetime
import os
import pandas as pd
load_dotenv()

api_key = os.getenv('BINANCE_API_KEY')  
api_secret = os.getenv('BINANCE_API_SECRET')
client = Client(api_key, api_secret)

# Get order book data for BTC/USDT pair
symbol = 'PENDLEUSDT'
depth = client.get_order_book(symbol=symbol)

# # Convert to DataFrames
bids_df = pd.DataFrame(depth['bids'], columns=['price', 'quantity'])
asks_df = pd.DataFrame(depth['asks'], columns=['price', 'quantity'])

# Convert string values to float
bids_df = bids_df.astype(float)
asks_df = asks_df.astype(float)

print(f"\nBids:\n{bids_df.head(10)}")
print(f"\nAsks:\n{asks_df.head(10)}")


In [ ]:
def fetch_options_data(client, symbol, option_type=None):
    """
    Fetch options data for a given symbol and option type.
    
    Args:
        client: Binance client instance
        symbol: Trading pair symbol (e.g., 'BTCUSDT')
        option_type: 'PUT' or 'CALL' or None for both
    
    Returns:
        DataFrame containing the options data
    """
    current_time = datetime.now()
    print(f"Fetching options data for {symbol} at {current_time}")
    
    # Get options exchange info
    options_info = client.options_exchange_info()
    
    # Filter by underlying asset
    filtered_options = [i for i in options_info['optionSymbols'] if i['underlying'] == symbol]
    
    # Filter by option type if specified
    if option_type:
        filtered_options = [i for i in filtered_options if i['side'] == option_type]
    
    # Convert timestamp to readable date format
    for option in filtered_options:
        option['expiryDate'] = datetime.fromtimestamp(option['expiryDate']/1000).strftime('%Y-%m-%d')
    
    # Create DataFrame for better presentation
    options_df = pd.DataFrame(filtered_options)
    
    # Display available expiry dates
    expiry_dates = set(options_df['expiryDate'])
    print(f"Available expiry dates: {expiry_dates}")
    
    return options_df



# Fetch ETH PUT options
eth_options = fetch_options_data(client, 'BTCUSDT', option_type='CALL')
eth_options['maxPrice'] = eth_options['filters'].apply(lambda x: x[0]['maxPrice'])
date = '2025-06-27'
# Display the options in a more readable format
# You can customize the columns to display
display_columns = ['symbol', 'underlying', 'side', 'expiryDate', 'strikePrice', 'maxPrice']
eth_options[eth_options.expiryDate == date][display_columns].sort_values('strikePrice')
